### PARTICIONES

In [0]:
# Los datos se dividen en particiones que se distribuyen en la memoria de distintos nodos (nodos workers)

# Se intenta que dicho reparto se efectúe de la forma más óptima posible

# El Shuffle (movimiento de datos en el clúster) puede llegar a ser un problema en cuanto a optimización

# En PySpark, 'coalesce' y 'repartition' son operaciones que permiten reorganizar la distribución de los datos en los nodos del clúster.

In [0]:
# Se utiliza para reducir el número de particiones de un RDD (a las que indiques) y crea un RDD nuevo (suele ser más eficiente).
# Minimiza el movimiento de datos; pero no obtiene particiones homogéneas.

rdd = sc.parallelize([3, 2, 1, 4, 5, 7, 9, 6, 2, 8], 4)
rdd2 = rdd.coalesce(3)

    # Informativo del resultado
print(rdd2.getNumPartitions())  # Devuelve número de particiones
print(rdd2.glom().collect())    # Crea RDD cuyos elementos son las particiones

3
[[3, 2], [1, 4], [5, 7, 9, 6, 2, 8]]


In [0]:
# Puede aumentar o reducir el número de  particiones del RDD (crea uno nuevo).
# Particiones resultantes son de igual tamaño (aprox.), pero produce shuffle

rdd = sc.parallelize([4, 5, 7, 9, 6, 2, 8, 3], 3)
rdd2 = rdd.repartition(4)

    # Informativo del resultado
print(rdd2.getNumPartitions())  # Devuelve número de particiones
print(rdd2.glom().collect())    # Crea RDD cuyos elementos son las particiones

4
[[6, 2, 8, 3], [7, 9], [], [4, 5]]


In [0]:
# Al final la diferencia radica en que hay dos variables a optimizar:
    # Homogeneidad de las particiones (más homogéneo, mejor)
    # Shuffle: movimiento de datos en el clúster (menos movimiento, mejor)
    
# Coalesce: Menos homogéneo (malo), menos shuffle (bueno)

# Repartition: Más homogéneo (bueno), más shuffle (malo)

In [0]:
# Crear un RDD a partir de un archivo
rdd = sc.parallelize([3, 2, 1, 4, 5, 7, 9, 6, 2, 8])

# Contar el número de líneas antes de la reducción de particiones
print("Antes de reducir particiones:", rdd.getNumPartitions())

# Reducir el número de particiones utilizando coalesce
rdd_coalesced = rdd.coalesce(2)

# Contar el número de líneas después de la reducción de particiones
print("Después de reducir particiones con coalesce:", rdd_coalesced.getNumPartitions())

# Redistribuir los datos utilizando repartition
rdd_repartitioned = rdd.repartition(4)

# Contar el número de líneas después de la redistribución de particiones
print("Después de redistribuir particiones con repartition:", rdd_repartitioned.getNumPartitions())


Antes de reducir particiones: 8
Después de reducir particiones con coalesce: 2
Después de redistribuir particiones con repartition: 4


### PERSISTENCIA

In [0]:
# La persistencia en PySpark se refiere a la capacidad de almacenar en caché los resultados de una operación en memoria o disco para su reutilización posterior. Mejora el rendimiento. Es decir, en lugar de calcularlo para cada vez que se pide, guarda el resultado y lo da tal cual cuando se pide.

# Se puede utilizar persist() o cache().

# Liberar la memoria caché: .unpersist() cuando ya no sea necesario.

In [0]:
# '<>' puede ser:
    # MEMORY_ONLY (por defecto)
    # DISK_ONLY (solo disco)
    # MEMORY_AND_DISK (1º memoria, y si no cabe, disco)

from pyspark import StorageLevel

file = 'dbfs:/FileStore/shared_uploads/danielmm97@gmail.com/quijote.txt'
lineas = sc.textFile(file)
long_lineas = lineas.map(lambda elemento: len(elemento))
long_lineas.persist(StorageLevel.MEMORY_ONLY) 				

print(long_lineas.reduce(lambda elem1, elem2: elem1 + elem2))

2034318


In [0]:
# cache() es una versión abreviada de persist() porque tiene el parámetro predeterminado seteado en MEMORY_ONLY.
    # rdd.persist(StorageLevel.MEMORY_ONLY) == rdd.cache()

# Si se desea especificar un nivel de almacenamiento diferente a MEMORY_ONLY, no puede usarse cache() es necesario utilizar persist(...) con el argumento correspondiente.

file = 'dbfs:/FileStore/shared_uploads/danielmm97@gmail.com/quijote.txt'
lineas = sc.textFile(file)
long_lineas = lineas.flatMap(lambda elemento: elemento.split())
long_lineas.cache() 
 
print(long_lineas.count())


376500
